<a href="https://colab.research.google.com/github/AdrianoCorreia94/csv/blob/master/SPRINT07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL ARQUIVO NOMES.CSV




In [ ]:
# opcao para importar arquivo nomes.csv

# via pandas
#import pandas as pd

# criar df pandas
#df = pd.read_csv('https://raw.githubusercontent.com/AdrianoCorreia94/csv/master/nomes.csv')

# transformar df pandas em df spark
#df_spark = spark.createDataFrame(df)
#df_spark.show()

# via terminal
#!wget https://raw.githubusercontent.com/AdrianoCorreia94/csv/master/nomes.csv


In [ ]:
!pip install pyspark py4j
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("teste").getOrCreate()


In [187]:
## 1 Ler o arquivo nomes.csv no S3 (lembre-se de realizar upload do arquivo antes).
dataset = spark.read.csv("/content/nomes.csv", header=True, sep=",", inferSchema=True)

In [ ]:
## 2 Imprima o schema do dataframe gerado no passo anterior.
dataset.schema

StructType([StructField('nome', StringType(), True), StructField('sexo', StringType(), True), StructField('total', IntegerType(), True), StructField('ano', IntegerType(), True)])

In [ ]:
## 3  Escrever o código necessário para alterar a caixa dos valores da coluna nome para MAIÚSCULO.
nome_maiusculo = dataset.withColumn("nome", upper(col("nome")))
nome_maiusculo.show()

+---------+----+-----+----+
|     nome|sexo|total| ano|
+---------+----+-----+----+
| JENNIFER|   F|54336|1983|
|  JESSICA|   F|45278|1983|
|   AMANDA|   F|33752|1983|
|   ASHLEY|   F|33292|1983|
|    SARAH|   F|27228|1983|
|  MELISSA|   F|23472|1983|
|   NICOLE|   F|22392|1983|
|STEPHANIE|   F|22323|1983|
|  HEATHER|   F|20749|1983|
|ELIZABETH|   F|19838|1983|
|  CRYSTAL|   F|17904|1983|
|      AMY|   F|17095|1983|
| MICHELLE|   F|16828|1983|
|  TIFFANY|   F|15960|1983|
| KIMBERLY|   F|15374|1983|
|CHRISTINA|   F|15359|1983|
|    AMBER|   F|14886|1983|
|     ERIN|   F|14835|1983|
|  REBECCA|   F|14711|1983|
|   RACHEL|   F|14588|1983|
+---------+----+-----+----+
only showing top 20 rows



In [ ]:
## 4 Imprimir a contagem de linhas presentes no dataframe.
print('contagem de linhas')
dataset.count()

contagem de linhas


1825433

In [ ]:
## 5 Imprimir a contagem de nomes, agrupando os dados do dataframe pelas colunas ano e sexo.
contagem_nomes = dataset.groupBy("ano", "sexo").agg(count("*").alias("contagem"))
contagem_nomes.show()

+----+----+--------+
| ano|sexo|contagem|
+----+----+--------+
|2004|   F|   18819|
|1882|   F|    1028|
|1906|   M|    1413|
|1908|   M|    1584|
|1893|   M|    1179|
|1885|   F|    1197|
|2006|   F|   20043|
|1992|   F|   15605|
|1889|   F|    1479|
|1880|   M|    1058|
|1992|   M|    9811|
|1896|   F|    1825|
|1986|   M|    7823|
|1905|   F|    2235|
|2001|   M|   12295|
|2008|   M|   14606|
|2011|   F|   19540|
|1892|   M|    1260|
|1907|   F|    2399|
|1907|   M|    1549|
+----+----+--------+
only showing top 20 rows



In [ ]:
# 6 Ordene os dados de modo que o ano mais recente apareça como primeiro registro do dataframe.
ordenado = dataset.orderBy(col("ano").desc())
ordenado.show()

+---------+----+-----+----+
|     nome|sexo|total| ano|
+---------+----+-----+----+
|     Emma|   F|20799|2014|
|    Grace|   F| 7554|2014|
|   Olivia|   F|19674|2014|
|   Sophia|   F|18490|2014|
| Isabella|   F|16950|2014|
|      Ava|   F|15586|2014|
|      Mia|   F|13442|2014|
|    Emily|   F|12562|2014|
|  Abigail|   F|11985|2014|
|  Madison|   F|10247|2014|
|Charlotte|   F|10048|2014|
|   Harper|   F| 9564|2014|
|    Sofia|   F| 9542|2014|
|    Avery|   F| 9517|2014|
|Elizabeth|   F| 9492|2014|
|   Amelia|   F| 8727|2014|
|   Evelyn|   F| 8692|2014|
|     Ella|   F| 8489|2014|
|    Chloe|   F| 8469|2014|
| Victoria|   F| 7955|2014|
+---------+----+-----+----+
only showing top 20 rows



In [ ]:
# 7 Apresentar qual foi o nome feminino com mais registros e em que ano ocorreu.

# filtrar por nome feminino
nomes_femininos = dataset.filter(col("sexo") == "F")

# ordenar por total
feminino_mais_registrado = nomes_femininos.orderBy(col("total").desc())

# apresentar o maior
feminino_mais_registrado.first()


Row(nome='Linda', sexo='F', total=99680, ano=1947)

In [ ]:
# 8 Apresentar qual foi o nome masculino com mais registros e em que ano ocorreu.

# filtrar por sexo
nomes_masculinos = dataset.filter(col("sexo") == "M")

# ordenar descrescente por total
masculino_mais_registrado = nomes_masculinos.orderBy(col("total").desc())

# apresentar o maior
masculino_mais_registrado.first()

Row(nome='James', sexo='M', total=94755, ano=1947)

In [ ]:
# 9 Apresentar o total de registros (masculinos e femininos) para cada ano presente no dataframe.
# Considere apenas as primeiras 10 linhas, ordenadas pelo ano, de forma crescente.

# agrupar por ano e sexo e incluir o total
total_registros = dataset.groupBy("ano", "sexo").agg(functions.count("*").alias("total"))

# ordenar por anoo de forma crescente
total_ordenado = total_registros.orderBy(col("ano")).limit(10)

# apresentar
total_ordenado.show()

+----+----+-----+
| ano|sexo|total|
+----+----+-----+
|1880|   F|  942|
|1880|   M| 1058|
|1881|   M|  997|
|1881|   F|  938|
|1882|   F| 1028|
|1882|   M| 1099|
|1883|   M| 1030|
|1883|   F| 1054|
|1884|   F| 1172|
|1884|   M| 1125|
+----+----+-----+



In [ ]:
# 10 Escrever o conteúdo do dataframe com os valores de nome em maiúsculo no S3.

# 10.01 criar particao
particao = nome_maiusculo.write.partitionBy('sexo','ano').saveAsTable('particao')

In [ ]:
# 10.02 testar particao
spark.sql("select * from particao").show()

In [ ]:
# 10.03 criar as credenciais aws de forma externa em um arquivo json
# por opcao nao sera testado este codigo devido a seguranca das credenciais aws


# importar json
import json


# Ler as configurações do arquivo JSON
with open("config.json", "r") as config_file:
    config_data = json.load(config_file)

# Acessando as configurações de login e senha aws pelo json
login = config_data.get("aws_access_key_id")
pwd = config_data.get("aws_secret_access_key")

# testar a leitura das configs
print(login, pwd)


login de acesso senha de acesso


In [ ]:
# 10.04 Escrever o conteúdo do dataframe com os valores de nome em maiúsculo no S3.
particao.write.mode('overwrite').json('s3://adrianoguedin/lab-glue/frequencia_registro_nomes_eua/')

In [ ]:
# Atenção aos requisitos:
# A gravação deve ocorrer no subdiretório frequencia_registro_nomes_eua do path s3://<BUCKET>/lab-glue/
# O formato deve ser JSON
# O particionamento deverá ser realizado pelas colunas sexo e ano (nesta ordem)